In [1]:
from __future__ import print_function

## #1

In [2]:
import torch

In [6]:
x = torch.Tensor(5, 3)
print(x)


1.00000e-10 *
 -2.0019  0.0000 -2.0019
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
[torch.FloatTensor of size 5x3]



In [7]:
x = torch.rand(5, 3)
print(x)


 0.7742  0.9414  0.8177
 0.9527  0.0556  0.6209
 0.7736  0.5717  0.5081
 0.8508  0.2422  0.5328
 0.7847  0.0893  0.9592
[torch.FloatTensor of size 5x3]



In [8]:
print(x.size())

torch.Size([5, 3])


In [9]:
y = torch.rand(5, 3)
print(x + y)


 1.5710  1.4593  1.6078
 1.8805  0.3010  0.7918
 1.4932  1.1798  1.2699
 1.1985  0.2619  1.4863
 1.2069  0.8088  1.6696
[torch.FloatTensor of size 5x3]



In [11]:
print(torch.add(x, y))


 1.5710  1.4593  1.6078
 1.8805  0.3010  0.7918
 1.4932  1.1798  1.2699
 1.1985  0.2619  1.4863
 1.2069  0.8088  1.6696
[torch.FloatTensor of size 5x3]



In [13]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)


 1.5710  1.4593  1.6078
 1.8805  0.3010  0.7918
 1.4932  1.1798  1.2699
 1.1985  0.2619  1.4863
 1.2069  0.8088  1.6696
[torch.FloatTensor of size 5x3]

In [27]:
y.add(x)


 6.2164  7.1079  6.5137
 7.5968  0.6346  4.5169
 6.1348  4.6099  4.3185
 6.3033  1.7152  4.6832
 5.9150  1.3444  7.4246
[torch.FloatTensor of size 5x3]

In [28]:
print(x[:, 1])


 0.9414
 0.0556
 0.5717
 0.2422
 0.0893
[torch.FloatTensor of size 5]



In [32]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [44]:
x.view(2, -1)


 1.3141 -0.0740 -0.0350 -0.2393  1.5021  0.6207 -0.2963 -1.0127
-0.8536 -0.1702  0.7754  0.4832 -0.0540 -0.2032  0.0480  0.4855
[torch.FloatTensor of size 2x8]

In [46]:
a = torch.ones(5)
print(a)
b = a.numpy()
print(b)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

[ 1.  1.  1.  1.  1.]


In [47]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[ 2.  2.  2.  2.  2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]



In [48]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

## #2

In [49]:
from torch.autograd import Variable

In [55]:
x = Variable(torch.ones(2, 2), requires_grad=True)

In [57]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [58]:
print(y.grad_fn)

In [59]:
z = y * y * 3
out = z.mean()

print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



In [60]:
out.backward()

In [64]:
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]



In [85]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

Variable containing:
 1382.2461
  199.2658
  910.1483
[torch.FloatTensor of size 3]



In [86]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



In [87]:
import torch.nn as nn
import torch.nn.functional as F

In [148]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(nn.Linear(self.num_flat_features(x), 120)(x))
#         x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [149]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [150]:
input = Variable(torch.randn(1, 1, 40, 38))
out = net(input)
print(out)

Variable containing:
1.00000e-02 *
  6.6850 -2.8187 -6.4826  7.7523 -8.2894 -1.5609  1.4571  8.2110 -3.9600 -5.1245
[torch.FloatTensor of size 1x10]



In [156]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [104]:
output = net(input)
target = Variable(torch.arange(1, 11))
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.4772
[torch.FloatTensor of size 1]



In [105]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [154]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
None


In [107]:
x[1:]

Variable containing:
 0.3892
 1.7776
[torch.FloatTensor of size 2]

In [110]:
x.size()[1:]

torch.Size([])

In [144]:
output = net(input)
target = Variable(torch.arange(1, 11))
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.1413
[torch.FloatTensor of size 1]



In [145]:
torch.arange(1, 11)


  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
[torch.FloatTensor of size 10]

In [157]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

AttributeError: 'NoneType' object has no attribute 'data'

In [158]:
import torch.optim as optim

In [159]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [160]:
optimizer.zero_grad()

In [161]:
output = net(input)

In [162]:
loss = criterion(output, target)
loss.backward()
optimizer.step()

In [163]:
loss

Variable containing:
 38.7699
[torch.FloatTensor of size 1]